In [1]:
import collections
import pandas as pd
import pyreadr
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from venn import venn
import numpy as np
import math
import scipy

/var/folders/y8/66w8_8kj0dx2znq80cdbl_rw0000gn/T/ipykernel_37300/2185171821.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
clinical_df = pyreadr.read_r('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/figurecode/data/20221109_TRACERx421_all_patient_df.rds')
clinical_df = pd.DataFrame(clinical_df[None])
clinical_df.head()

,cruk_id,tumour_id_muttable_cruk,tumour_id_per_patient,age,sex,ethnicity,cigs_perday,years_smoking,packyears,smoking_status_merged,...,os_time,cens_dfs,dfs_time,cens_dfs_any_event,dfs_time_any_event,cens_lung_event,lung_event_time,Relapse_cat,Relapse_cat_new,tx100
0,CRUK0034,CRUK0034,CRUK0034,68,Female,White- Irish,20.0,35,35.000,Ex-Smoker,...,1849,0,1849,0,1849,0,1849,NaN,No rec,True
1,CRUK0150,CRUK0150,CRUK0150,81,Male,White- British,44.5,49,109.025,Ex-Smoker,...,1362,1,1362,1,1362,0,1362,NaN,No rec,False
2,CRUK0159,CRUK0159,CRUK0159,60,Male,White- British,20.0,38,38.000,Smoker,...,2224,1,2224,1,1935,0,2224,Second primary lung,No rec,False
3,CRUK0090,CRUK0090,CRUK0090,65,Male,White- British,10.0,35,17.500,Ex-Smoker,...,2365,1,1143,1,1143,1,1143,Intrathoracic,Intrathoracic,True
4,CRUK0045,CRUK0045,CRUK0045,85,Male,White- British,10.0,25,12.500,Ex-Smoker,...,986,1,917,1,917,1,917,Intrathoracic,Intrathoracic,True


In [3]:
tumor_df = pyreadr.read_r('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/LCCE_converted_data/20221109_TRACERx421_all_tumour_df.rds')
tumor_df = pd.DataFrame(tumor_df[None])
tumor_df.head()

,tumour_id,patient_id,clinical_sex,age,ethnicity,Histology_per_tumour_id_muttable,histology_3,LUAD_pred_subtype_with.IMA_per_tumour,site_per_lesion,pT_stage_per_lesion,pN_stage_per_lesion,size_pathology_per_lesion,vascular_invasion_per_lesion,pleural_invasion_per_lesion,smoking_status_merged,cigs_perday,years_smoking,pack_years,is.family.lung,pathologyTNM
0,CRUK0034,CRUK0034,Female,68,White- Irish,Invasive adenocarcinoma,LUAD,acinar,Right Upper Lobe,2a,0,40,Yes,No,Ex-Smoker,20.0,35,35.000,True,IB
1,CRUK0150,CRUK0150,Male,81,White- British,LCNEC,Other,NaN,Left Upper Lobe,3,0,75,Yes,No,Ex-Smoker,44.5,49,109.025,False,IIB
2,CRUK0159,CRUK0159,Male,60,White- British,Invasive adenocarcinoma,LUAD,acinar,Right Upper Lobe,2a,0,36,Yes,Yes,Smoker,20.0,38,38.000,False,IB
3,CRUK0090,CRUK0090,Male,65,White- British,Squamous cell carcinoma,LUSC,NaN,Right Upper Lobe,2a,1,35,Yes,Yes,Ex-Smoker,10.0,35,17.500,False,IIA
4,CRUK0045,CRUK0045,Male,85,White- British,Invasive adenocarcinoma,LUAD,papillary,Left Lower Lobe,2a,0,48,Yes,Yes,Ex-Smoker,10.0,25,12.500,False,IB


In [4]:
mut_table = pyreadr.read_r('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/LCCE_converted_data/20221109_TRACERx421_mutation_table.rds')
mut_table = pd.DataFrame(mut_table[None])
mut_table.head()

,patient_id,tumour_id,mutation_id,chr,start,stop,ref,var,Hugo_Symbol,func,...,AAChange,GL_VAF,GL_nAlt,GL_depth,ITHState,PyCloneClonal_SC,PyCloneCluster_SC,cleanCluster_SC,combTiming_SC,DriverMut
0,CRUK0005,CRUK0005,CRUK0005:19:34291428:G,chr19,34291428,34291428,G,T,KCTD15,intronic,...,NaN,0,0,187,1,S,3.0,True,subclonal,False
1,CRUK0005,CRUK0005,CRUK0005:19:9084770:C,chr19,9084770,9084770,C,A,MUC16,exonic,...,p.V2349L,0.39,2,510,1,C,2.0,True,early,False
2,CRUK0005,CRUK0005,CRUK0005:1:2160418:G,chr1,2160418,2160418,G,T,SKI,exonic,...,p.P71P,0,0,114,1,C,2.0,True,early,False
3,CRUK0005,CRUK0005,CRUK0005:19:40095981:G,chr19,40095981,40095981,G,T,LGALS13,exonic,...,p.G86C,0,0,659,1,C,2.0,True,early,False
4,CRUK0005,CRUK0005,CRUK0005:19:41313703:C,chr19,41313703,41313703,C,G,RAB4B-EGLN2,ncRNA_intronic,...,NaN,0,0,894,3,S,5.0,True,subclonal,False


In [5]:
clinical_df = clinical_df.sort_values(by=['cruk_id'])
clinical_df

,cruk_id,tumour_id_muttable_cruk,tumour_id_per_patient,age,sex,ethnicity,cigs_perday,years_smoking,packyears,smoking_status_merged,...,os_time,cens_dfs,dfs_time,cens_dfs_any_event,dfs_time_any_event,cens_lung_event,lung_event_time,Relapse_cat,Relapse_cat_new,tx100
88,CRUK0001,CRUK0001,CRUK0001,84,Male,White- British,20.0,30,30.00,Ex-Smoker,...,1120,1,1120,1,1120,0,1120,NaN,No rec,True
79,CRUK0002,CRUK0002,CRUK0002,81,Male,White- British,5.0,35,8.75,Ex-Smoker,...,1901,0,1901,0,1901,0,1901,NaN,No rec,True
26,CRUK0003,CRUK0003,CRUK0003,78,Female,White- British,0.0,0,0.00,Never Smoked,...,2304,1,217,1,217,1,217,Intra & Extra,Intra & Extra,True
23,CRUK0004,CRUK0004,CRUK0004,69,Female,White- British,10.0,27,13.50,Ex-Smoker,...,1323,1,325,1,325,1,325,Intra & Extra,Intra & Extra,True
7,CRUK0005,CRUK0005,CRUK0005,74,Female,White- British,20.0,56,56.00,Ex-Smoker,...,649,1,649,1,649,0,649,NaN,No rec,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414,CRUK0861,CRUK0861,CRUK0861,65,Female,Middle eastern,10.0,45,22.50,Smoker,...,1084,0,1084,0,1084,0,1084,NaN,No rec,False
418,CRUK0864,CRUK0864,CRUK0864,78,Female,White- British,10.0,30,15.00,Ex-Smoker,...,1434,0,1434,0,1434,0,1434,NaN,No rec,False
416,CRUK0872,CRUK0872,CRUK0872,66,Male,White- British,20.0,54,54.00,Ex-Smoker,...,893,1,883,1,883,1,883,Intra & Extra,Intra & Extra,False
420,CRUK0881,CRUK0881_Tumour1,CRUK0881_Tumour1;CRUK0881_Tumour2,84,Male,White- British,20.0,70,70.00,Smoker,...,758,1,748,1,748,1,748,Intrathoracic,Intrathoracic,False


In [6]:
# Making the training data

y_train = []
for index, row in clinical_df.iterrows():
    if row['first_dfs_any_event_rec.or.new.primary'] == 'recurrence':
        y_train.append(1)
    else:
        y_train.append(0)

y = np.array(y_train)
print(y)

[0 0 1 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 1 0 1 0 0 1 1 1 1
 1 0 1 0 0 1 1 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 1 1 0
 0 1 0 0 1 0 0 0 0 0 1 1 0 1 0 1 0 1 0 1 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1
 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 1 0 0 0 1 1 1 0 1 1 1 1 1 1 0 0 0 0 0 1
 1 0 0 0 0 0 0 1 0 1 0 1 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 1 1 0
 0 0 0 1 1 0 1 0 1 0 1 0 1 1 0 0 0 0 0 0 1 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0
 1 1 0 0 1 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 1
 0 1 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 1 0 0 0 1 1 0 0 0 1 0 0 1 1 1 0 1
 0 0 0 1 0 0 0 1 1 1 0 0 1 0 1 0 0 0 0 0 0 0 1 1 1 0 0 1 0 0 0 0 0 0 0 1 0
 1 0 0 1 1 0 0 0 0 1 0 1 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1
 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 1 1 0 0 1 0 0 0 1
 0 1 0 0 0 0 0 1 1 0 0 1 1 0]


In [7]:
# read table into memory
cna_table = pyreadr.read_r('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/LCCE_converted_data/20221109_TRACERx421_scna_table.rds')
# Obtan dataframe and rename column sample to RegionID
cna_table = pd.DataFrame(cna_table[None])
cna_table.head()

,patient_id,tumour_id,RegionID,chr,startpos,endpos,n.het,cnTotal,nMajor,nMinor,Ploidy,ACF,nAraw,nBraw,cpn_event_vs_ploidy,MSAI,samp_used_to_phase
0,CRUK0005,CRUK0005,CRUK0005:R1,1,762273,2252860,73,3,2,1,3.131282,0.15,2.463756,1.184761,neutral,False,CRUK0005_SU_T1-R2
1,CRUK0005,CRUK0005,CRUK0005:R1,1,2252970,2527412,26,2,2,0,3.131282,0.15,1.645818,0.284001,neutral,False,CRUK0005_SU_T1-R2
2,CRUK0005,CRUK0005,CRUK0005:R1,1,2527437,2700215,12,3,2,1,3.131282,0.15,2.187291,0.956066,neutral,False,CRUK0005_SU_T1-R3
3,CRUK0005,CRUK0005,CRUK0005:R1,1,2700332,28531815,600,3,2,1,3.131282,0.15,1.725844,0.708290,neutral,False,CRUK0005_SU_T1-R2
4,CRUK0005,CRUK0005,CRUK0005:R1,1,28534782,29649999,11,3,2,1,3.131282,0.15,1.711109,0.695610,neutral,False,CRUK0005_SU_T1-R2


In [8]:
gene_matrix = pd.read_csv('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/Matrices/100_gene_matrix.csv')
driver_matrix = pd.read_csv('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/Matrices/100_driver_matrix.csv')
fifty_gene_matrix = pd.read_csv('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/Matrices/50_gene_matrix.csv')
fifty_driver_matrix = pd.read_csv('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/Matrices/50_driver_matrix.csv')
twenty_gene_matrix = pd.read_csv('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/Matrices/20_gene_matrix.csv')
twenty_driver_matrix = pd.read_csv('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/Matrices/20_driver_matrix.csv')

In [9]:
cna_table_no_duplicates = cna_table.drop_duplicates(subset='patient_id')
cna_table_no_duplicates

,patient_id,tumour_id,RegionID,chr,startpos,endpos,n.het,cnTotal,nMajor,nMinor,Ploidy,ACF,nAraw,nBraw,cpn_event_vs_ploidy,MSAI,samp_used_to_phase
0,CRUK0005,CRUK0005,CRUK0005:R1,1,762273,2252860,73,3,2,1,3.131282,0.15,2.463756,1.184761,neutral,False,CRUK0005_SU_T1-R2
1260,CRUK0057,CRUK0057,CRUK0057:R1,1,762273,10523520,175,2,1,1,3.870640,0.22,1.280482,1.116775,loss,False,CRUK0057_SU_T1-R2
1686,CRUK0039,CRUK0039,CRUK0039:R1,1,762273,934385,17,2,2,0,3.151741,0.26,2.079355,0.004553,loss,False,CRUK0039_SU_T1-R1
2090,CRUK0196,CRUK0196,CRUK0196:R1,1,762273,3301721,85,4,2,2,3.822113,0.21,1.876121,1.679011,neutral,NaN,none_no_allelic_imbalance_called
2633,CRUK0023,CRUK0023,CRUK0023:R1,1,762273,3394674,127,2,1,1,2.231459,0.60,1.088370,1.082848,neutral,False,CRUK0023_SU_T1-R3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
410988,CRUK0030,CRUK0030_Tumour2,:R1,1,69511,3646137,106,2,1,1,1.882182,0.18,1.076584,0.915059,neutral,NaN,none_no_allelic_imbalance_called
411579,CRUK0704,CRUK0704_Tumour1,:R1,1,762273,1387531,44,4,2,2,3.507856,0.43,2.020218,2.012841,neutral,False,CRUK0704_SU_T2-R4
416583,CRUK0372,CRUK0372_Tumour1,:R1,1,752894,1374834,76,1,1,0,2.016251,0.14,1.022760,0.447565,neutral,False,CRUK0372_SU_T2-R1
417335,CRUK0881,CRUK0881_Tumour1,:R1,1,65797,1027553,39,3,2,1,4.186610,0.33,1.992436,0.923056,neutral,False,CRUK0881_SU_T1-R5


In [10]:
gene_matrix_w_ploidy = gene_matrix.copy()
driver_matrix_w_ploidy = driver_matrix.copy()

# Add ploidy to gene_matrix and driver_matrix
gene_matrix_w_ploidy['Ploidy'] = gene_matrix_w_ploidy['patient_id'].map(cna_table_no_duplicates.set_index('patient_id')['Ploidy'])
driver_matrix_w_ploidy['Ploidy'] = driver_matrix_w_ploidy['patient_id'].map(cna_table_no_duplicates.set_index('patient_id')['Ploidy'])

In [11]:
gene_matrix_w_ploidy

,patient_id,ABCA13,ADAMTS12,ADAMTS16,ADAMTS20,AFF2,ANK2,APOB,ASTN1,BAI3,...,TTC40,TTN,UNC79,UNC80,USH2A,WDFY4,XIRP2,ZFHX4,ZNF536,Ploidy
0,CRUK0001,1,1,0,1,1,1,1,0,1,...,0,1,0,1,1,1,0,0,1,4.493770
1,CRUK0002,0,0,0,1,0,0,0,1,0,...,0,1,0,0,0,1,0,0,0,1.921907
2,CRUK0003,0,0,1,0,0,0,0,0,0,...,0,1,1,0,1,0,0,0,0,3.567392
3,CRUK0004,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,2.950603
4,CRUK0005,1,1,0,1,1,1,1,0,0,...,0,1,0,1,1,1,1,1,1,3.131282
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,CRUK0861,0,0,0,0,1,1,1,0,1,...,0,1,0,0,0,0,0,0,0,3.514533
417,CRUK0864,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,3.556913
418,CRUK0872,0,0,0,0,0,0,1,1,0,...,0,1,0,1,1,0,0,1,0,3.919491
419,CRUK0881,0,1,1,1,0,1,1,1,0,...,1,1,0,1,1,1,1,1,0,4.186610


In [12]:
driver_matrix_w_ploidy

,patient_id,ACVR1B,AMER1,APC,ARHGAP35,ARID1A,ARID1B,ARID2,ASXL1,ASXL2,...,STK11,TET2,TP53,TP53BP1,TSC2,UBR5,WT1,ZFP36L1,ZMYM2,Ploidy
0,CRUK0001,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,4.493770
1,CRUK0002,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.921907
2,CRUK0003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3.567392
3,CRUK0004,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,2.950603
4,CRUK0005,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,3.131282
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,CRUK0861,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,3.514533
417,CRUK0864,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3.556913
418,CRUK0872,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,3.919491
419,CRUK0881,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,4.186610


In [13]:
ploidy_na = gene_matrix_w_ploidy[gene_matrix_w_ploidy['Ploidy'].isna()]

In [14]:
ploidy_na = driver_matrix_w_ploidy[driver_matrix_w_ploidy['Ploidy'].isna()]

In [15]:
ploidy_na['patient_id']

6      CRUK0007
40     CRUK0042
53     CRUK0055
58     CRUK0060
86     CRUK0089
90     CRUK0093
99     CRUK0117
102    CRUK0128
113    CRUK0182
117    CRUK0199
119    CRUK0202
125    CRUK0227
156    CRUK0285
160    CRUK0293
164    CRUK0301
166    CRUK0305
167    CRUK0308
180    CRUK0325
196    CRUK0366
255    CRUK0467
257    CRUK0471
292    CRUK0540
309    CRUK0569
316    CRUK0581
318    CRUK0584
346    CRUK0643
406    CRUK0781
Name: patient_id, dtype: object

In [16]:
len(ploidy_na)

27

In [17]:
# Making the training data

clinical_df_ploidy = clinical_df.copy()
clinical_df_ploidy = clinical_df_ploidy.sort_values(by=['cruk_id'])
clinical_df_ploidy = clinical_df_ploidy[~clinical_df_ploidy['cruk_id'].isin(ploidy_na['patient_id'])]

y_train_ploidy = []
for index, row in clinical_df_ploidy.iterrows():
    if row['first_dfs_any_event_rec.or.new.primary'] == 'recurrence':
        y_train_ploidy.append(1)
    else:
        y_train_ploidy.append(0)

y_ploidy = np.array(y_train_ploidy)
print(y_ploidy)
print('length:',len(y_ploidy))

[0 0 1 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 1 0 1 0 0 1 1 1 1 1
 0 1 0 1 1 0 0 1 0 1 0 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 0 0 0 1 1 0 0 1 0 0
 1 0 0 0 0 0 1 1 1 0 1 1 0 1 1 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 1 1 0
 0 0 0 0 0 1 0 0 0 1 1 1 0 1 1 1 1 1 1 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 0 1 1
 0 1 1 0 1 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 1 1 0 1 0 1 0 1 1 1 0 0 0 0 0
 0 1 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 1 0 0 1 1 0 0 0 1 1 0 0 0 0 0 0 0 1
 0 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 1 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 1
 0 0 0 1 1 0 0 0 1 0 0 1 1 0 1 0 0 0 1 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 1 1
 0 0 1 0 0 0 0 0 0 0 1 0 1 0 0 1 1 0 0 0 0 1 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0
 0 0 0 0 0 1 0 0 1 0 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 0 0
 0 0 1 1 0 0 1 0 0 0 0 1 0 0 0 0 0 1 1 0 0 1 1 0]
length: 394


In [18]:
gene_matrix_w_ploidy = gene_matrix_w_ploidy[~gene_matrix_w_ploidy['patient_id'].isin(ploidy_na['patient_id'])]
gene_matrix_w_ploidy = gene_matrix_w_ploidy.set_index(gene_matrix_w_ploidy.columns[0])
gene_matrix_w_ploidy

,ABCA13,ADAMTS12,ADAMTS16,ADAMTS20,AFF2,ANK2,APOB,ASTN1,BAI3,CACNA1B,...,TTC40,TTN,UNC79,UNC80,USH2A,WDFY4,XIRP2,ZFHX4,ZNF536,Ploidy
patient_id,,,,,,,,,,,,,,,,,,,,,
CRUK0001,1,1,0,1,1,1,1,0,1,0,...,0,1,0,1,1,1,0,0,1,4.493770
CRUK0002,0,0,0,1,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,0,1.921907
CRUK0003,0,0,1,0,0,0,0,0,0,1,...,0,1,1,0,1,0,0,0,0,3.567392
CRUK0004,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,2.950603
CRUK0005,1,1,0,1,1,1,1,0,0,0,...,0,1,0,1,1,1,1,1,1,3.131282
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CRUK0861,0,0,0,0,1,1,1,0,1,0,...,0,1,0,0,0,0,0,0,0,3.514533
CRUK0864,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,3.556913
CRUK0872,0,0,0,0,0,0,1,1,0,0,...,0,1,0,1,1,0,0,1,0,3.919491


In [19]:
driver_matrix_w_ploidy = driver_matrix_w_ploidy[~driver_matrix_w_ploidy['patient_id'].isin(ploidy_na['patient_id'])]
driver_matrix_w_ploidy = driver_matrix_w_ploidy.set_index(driver_matrix_w_ploidy.columns[0])
driver_matrix_w_ploidy

,ACVR1B,AMER1,APC,ARHGAP35,ARID1A,ARID1B,ARID2,ASXL1,ASXL2,ATM,...,STK11,TET2,TP53,TP53BP1,TSC2,UBR5,WT1,ZFP36L1,ZMYM2,Ploidy
patient_id,,,,,,,,,,,,,,,,,,,,,
CRUK0001,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,4.493770
CRUK0002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.921907
CRUK0003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3.567392
CRUK0004,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,2.950603
CRUK0005,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,3.131282
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CRUK0861,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,3.514533
CRUK0864,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3.556913
CRUK0872,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,3.919491


In [20]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import GridSearchCV


X_train, X_test, y_train, y_test = train_test_split(gene_matrix_w_ploidy, y_ploidy, test_size = 0.4, random_state = 0, stratify = y_ploidy)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'C': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

lr_model = LogisticRegression(penalty='l2')

grid_search = GridSearchCV(lr_model, param_grid, scoring='balanced_accuracy')
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(236, 101) (158, 101) (236,) (158,)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://sciki

Best parameters:  {'C': 0.1}
Best score:  0.5145279886148008
Predictions:  [0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 1 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0
 0 0 0 0 0 0 0 0 0 0]
Actual:  [1 1 1 1 0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 1 1 1 0 0 1 1 0
 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1
 0 1 1 0 0 1 0 0 1 1 1 0 0 0 0 1 1 0 0 0 0 1 0 0 1 0 1 1 0 0 1 0 1 0 0 0 0
 0 0 1 1 1 1 1 1 1 0]
Balanced Accuracy:  0.5160256410256411
ROC AUC:  0.5735398860398861
               precision    recall  f1-score   support

No Recurrence       0.67      0.87      0.75       104
   Recurrence       0.39      0.17      0.23        54

     accuracy                        

from sklearn.metrics import roc_curve, roc_auc_score
y_score = lr_model.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_score, pos_label=1)
roc_auc = roc_auc_score(y_test, y_score)

plt.figure()
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')  # random predictions curve
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate or (1 - Specifity)')
plt.ylabel('True Positive Rate or (Sensitivity)')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [21]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

# Neural network/Multi-layer perceptron (MLP) algorithm, 100 mutations with ploidy

X_train, X_test, y_train, y_test = train_test_split(gene_matrix_w_ploidy, y_ploidy, test_size = 0.4, random_state = 0, stratify = y_ploidy)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'hidden_layer_sizes': [(1,), (2,), (5,), (10,), (15,)],
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

clf = MLPClassifier(solver='lbfgs', random_state=1, max_iter=2000)

grid_search = GridSearchCV(clf, param_grid, scoring='balanced_accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(236, 101) (158, 101) (236,) (158,)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_

Best parameters:  {'alpha': 1e-05, 'hidden_layer_sizes': (15,)}
Best score:  0.5842623339658444
Predictions:  [0 1 1 0 0 0 1 1 0 0 1 0 0 0 1 1 1 0 1 1 1 0 1 0 1 0 0 0 0 0 1 1 0 0 0 1 0
 1 1 0 1 0 0 1 0 0 1 0 0 1 1 1 1 0 0 1 0 1 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 1
 1 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 1 0 0 1 0 1 0 0 0 0 0 1 0 0 1 0 0 1 0 1
 0 1 1 0 1 0 0 1 1 1 1 1 1 0 1 0 1 1 0 1 1 1 1 0 1 1 0 1 1 0 0 1 0 0 1 1 0
 0 0 1 0 0 1 0 1 0 1]
Actual:  [1 1 1 1 0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 1 1 1 0 0 1 1 0
 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1
 0 1 1 0 0 1 0 0 1 1 1 0 0 0 0 1 1 0 0 0 0 1 0 0 1 0 1 1 0 0 1 0 1 0 0 0 0
 0 0 1 1 1 1 1 1 1 0]
Balanced Accuracy:  0.5432692307692308
ROC AUC:  0.5681089743589743
               precision    recall  f1-score   support

No Recurrence       0.69      0.59      0.64       104
   Recurrence       0.39      0.50      0.44        54

  

In [22]:
# Logsitic Regression, 100 driver mutations with ploidy

X_train, X_test, y_train, y_test = train_test_split(driver_matrix_w_ploidy, y_ploidy, test_size = 0.4, random_state = 0, stratify = y_ploidy)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'C': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

lr_model = LogisticRegression(penalty='l2')

grid_search = GridSearchCV(lr_model, param_grid, scoring='balanced_accuracy')
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(236, 101) (158, 101) (236,) (158,)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Best parameters:  {'C': 0.1}
Best score:  0.5092741935483871
Predictions:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0]
Actual:  [1 1 1 1 0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 1 1 1 0 0 1 1 0
 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1
 0 1 1 0 0 1 0 0 1 1 1 0 0 0 0 1 1 0 0 0 0 1 0 0 1 0 1 1 0 0 1 0 1 0 0 0 0
 0 0 1 1 1 1 1 1 1 0]
Balanced Accuracy:  0.4951923076923077
ROC AUC:  0.4170227920227921
               precision    recall  f1-score   support

No Recurrence       0.66      0.99      0.79       104
   Recurrence       0.00      0.00      0.00        54

     accuracy                        

from sklearn.metrics import roc_curve, roc_auc_score
y_score = lr_model.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_score, pos_label=1)
roc_auc = roc_auc_score(y_test, y_score)

plt.figure()
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')  # random predictions curve
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate or (1 - Specifity)')
plt.ylabel('True Positive Rate or (Sensitivity)')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [23]:
# Neural network/Multi-layer perceptron (MLP) algorithm, 100 driver mutations with ploidy

X_train, X_test, y_train, y_test = train_test_split(driver_matrix_w_ploidy, y_ploidy, test_size = 0.4, random_state = 0, stratify = y_ploidy)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'hidden_layer_sizes': [(1,), (2,), (5,), (10,), (15,)],
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

clf = MLPClassifier(solver='lbfgs', random_state=1, max_iter=2000)

grid_search = GridSearchCV(clf, param_grid, scoring='balanced_accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(236, 101) (158, 101) (236,) (158,)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network

Best parameters:  {'alpha': 1, 'hidden_layer_sizes': (1,)}
Best score:  0.5261503795066413
Predictions:  [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0
 0 1 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0
 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0]
Actual:  [1 1 1 1 0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 1 1 1 0 0 1 1 0
 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1
 0 1 1 0 0 1 0 0 1 1 1 0 0 0 0 1 1 0 0 0 0 1 0 0 1 0 1 1 0 0 1 0 1 0 0 0 0
 0 0 1 1 1 1 1 1 1 0]
Balanced Accuracy:  0.5156695156695157
ROC AUC:  0.5013354700854701
               precision    recall  f1-score   support

No Recurrence       0.67      0.85      0.75       104
   Recurrence       0.38      0.19      0.25        54

     ac

In [24]:
fifty_gene_matrix = fifty_gene_matrix.set_index(fifty_gene_matrix.columns[0])
fifty_driver_matrix = fifty_driver_matrix.set_index(fifty_driver_matrix.columns[0])
twenty_gene_matrix = twenty_gene_matrix.set_index(twenty_gene_matrix.columns[0])
twenty_driver_matrix = twenty_driver_matrix.set_index(twenty_driver_matrix.columns[0])

In [25]:
fifty_gene_matrix_w_ploidy = fifty_gene_matrix.copy()
fifty_driver_matrix_w_ploidy = fifty_driver_matrix.copy()

# Add ploidy to gene_matrix and driver_matrix
fifty_gene_matrix_w_ploidy['Ploidy'] = fifty_gene_matrix_w_ploidy.index.map(cna_table_no_duplicates.set_index('patient_id')['Ploidy'])
fifty_driver_matrix_w_ploidy['Ploidy'] = fifty_driver_matrix_w_ploidy.index.map(cna_table_no_duplicates.set_index('patient_id')['Ploidy'])

# Remove patients with N/A ploidy
fifty_gene_matrix_w_ploidy = fifty_gene_matrix_w_ploidy[~fifty_gene_matrix_w_ploidy.index.isin(ploidy_na['patient_id'])]
fifty_driver_matrix_w_ploidy = fifty_driver_matrix_w_ploidy[~fifty_driver_matrix_w_ploidy.index.isin(ploidy_na['patient_id'])]

print(fifty_gene_matrix_w_ploidy)
print(fifty_driver_matrix_w_ploidy)

            ABCA13  ANK2  APOB  CACNA1E  CCDC168  CNTNAP5  COL11A1  COL22A1  \
patient_id                                                                    
CRUK0001         1     1     1        0        0        1        0        1   
CRUK0002         0     0     0        0        0        0        0        0   
CRUK0003         0     0     0        0        0        0        0        0   
CRUK0004         1     0     0        0        1        0        0        0   
CRUK0005         1     1     1        1        1        0        1        1   
...            ...   ...   ...      ...      ...      ...      ...      ...   
CRUK0861         0     1     1        0        0        1        0        0   
CRUK0864         0     0     0        0        1        0        0        0   
CRUK0872         0     0     1        0        0        0        1        1   
CRUK0881         0     1     1        1        1        1        1        0   
CRUK0893         0     1     1        1        0    

In [26]:
twenty_gene_matrix_w_ploidy = twenty_gene_matrix.copy()
twenty_driver_matrix_w_ploidy = twenty_driver_matrix.copy()

# Add ploidy to gene_matrix and driver_matrix
twenty_gene_matrix_w_ploidy['Ploidy'] = twenty_gene_matrix_w_ploidy.index.map(cna_table_no_duplicates.set_index('patient_id')['Ploidy'])
twenty_driver_matrix_w_ploidy['Ploidy'] = twenty_driver_matrix_w_ploidy.index.map(cna_table_no_duplicates.set_index('patient_id')['Ploidy'])

# Remove patients with N/A ploidy
twenty_gene_matrix_w_ploidy = twenty_gene_matrix_w_ploidy[~twenty_gene_matrix_w_ploidy.index.isin(ploidy_na['patient_id'])]
twenty_driver_matrix_w_ploidy = twenty_driver_matrix_w_ploidy[~twenty_driver_matrix_w_ploidy.index.isin(ploidy_na['patient_id'])]

print(twenty_gene_matrix_w_ploidy)
print(twenty_driver_matrix_w_ploidy)

            COL22A1  CSMD1  CSMD3  DMD  LRP1B  MUC16  MUC19  NRXN1  OBSCN  \
patient_id                                                                  
CRUK0001          1      1      1    1      0      0      1      0      1   
CRUK0002          0      0      1    0      0      1      0      0      0   
CRUK0003          0      0      1    0      1      1      0      0      1   
CRUK0004          0      0      1    0      1      1      0      0      1   
CRUK0005          1      0      1    1      1      1      0      1      1   
...             ...    ...    ...  ...    ...    ...    ...    ...    ...   
CRUK0861          0      0      1    0      0      1      0      0      0   
CRUK0864          0      0      1    1      1      1      0      0      0   
CRUK0872          1      0      1    0      1      1      1      0      0   
CRUK0881          0      1      1    1      1      1      1      1      1   
CRUK0893          0      0      0    0      0      0      0      1      0   

In [27]:
ten_driver_matrix = pd.read_csv('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/Matrices/10_driver_matrix.csv')
ten_driver_matrix = ten_driver_matrix.set_index(ten_driver_matrix.columns[0])
ten_driver_matrix_w_ploidy = ten_driver_matrix.copy()

# Add ploidy to gene_matrix and driver_matrix
ten_driver_matrix_w_ploidy['Ploidy'] = ten_driver_matrix_w_ploidy.index.map(cna_table_no_duplicates.set_index('patient_id')['Ploidy'])

# Remove patients with N/A ploidy
ten_driver_matrix_w_ploidy = ten_driver_matrix_w_ploidy[~ten_driver_matrix_w_ploidy.index.isin(ploidy_na['patient_id'])]

print(ten_driver_matrix_w_ploidy)

            CDKN2A  FAT1  KEAP1  KMT2D  KRAS  NF1  PIK3CA  SMARCA4  STK11  \
patient_id                                                                  
CRUK0001         0     0      0      0     0    0       0        0      0   
CRUK0002         0     0      0      0     0    1       0        0      0   
CRUK0003         0     0      0      0     0    0       1        0      0   
CRUK0004         0     0      0      0     0    0       0        0      0   
CRUK0005         0     0      0      0     0    0       0        0      0   
...            ...   ...    ...    ...   ...  ...     ...      ...    ...   
CRUK0861         0     0      0      0     0    0       0        0      0   
CRUK0864         0     0      0      0     1    0       0        0      0   
CRUK0872         0     0      0      0     0    0       0        0      0   
CRUK0881         0     0      0      0     1    1       0        0      0   
CRUK0893         0     0      0      0     1    0       0        0      0   

In [29]:
# Logsitic Regression, 50 mutations with ploidy

X_train, X_test, y_train, y_test = train_test_split(fifty_gene_matrix_w_ploidy, y_ploidy, test_size = 0.4, random_state = 0, stratify = y_ploidy)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'C': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

lr_model = LogisticRegression(penalty='l2')

grid_search = GridSearchCV(lr_model, param_grid, scoring='balanced_accuracy')
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(236, 51) (158, 51) (236,) (158,)
Best parameters:  {'C': 0.1}
Best score:  0.5012452561669829
Predictions:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 1 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0]
Actual:  [1 1 1 1 0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 1 1 1 0 0 1 1 0
 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1
 0 1 1 0 0 1 0 0 1 1 1 0 0 0 0 1 1 0 0 0 0 1 0 0 1 0 1 1 0 0 1 0 1 0 0 0 0
 0 0 1 1 1 1 1 1 1 0]
Balanced Accuracy:  0.48896011396011396
ROC AUC:  0.5146011396011396
               precision    recall  f1-score   support

No Recurrence       0.65      0.90      0.76       104
   Recurrence       0.29      0.07      0.12        54

  

In [30]:
# Neural network/Multi-layer perceptron (MLP) algorithm, 50 mutations with ploidy

X_train, X_test, y_train, y_test = train_test_split(fifty_gene_matrix_w_ploidy, y_ploidy, test_size = 0.4, random_state = 0, stratify = y_ploidy)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'hidden_layer_sizes': [(1,), (2,), (5,), (10,), (15,)],
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

clf = MLPClassifier(solver='lbfgs', random_state=1, max_iter=2000)

grid_search = GridSearchCV(clf, param_grid, scoring='balanced_accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(236, 51) (158, 51) (236,) (158,)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py

Best parameters:  {'alpha': 1e-05, 'hidden_layer_sizes': (5,)}
Best score:  0.5424573055028463
Predictions:  [0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 1
 1 1 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 0 0
 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 1 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 1 0 0 0 1 0 1
 0 0 0 1 0 0 0 0 1 0]
Actual:  [1 1 1 1 0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 1 1 1 0 0 1 1 0
 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1
 0 1 1 0 0 1 0 0 1 1 1 0 0 0 0 1 1 0 0 0 0 1 0 0 1 0 1 1 0 0 1 0 1 0 0 0 0
 0 0 1 1 1 1 1 1 1 0]
Balanced Accuracy:  0.4861111111111111
ROC AUC:  0.48041310541310545
               precision    recall  f1-score   support

No Recurrence       0.65      0.75      0.70       104
   Recurrence       0.32      0.22      0.26        54

  

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [31]:
# Logsitic Regression, 50 driver mutations with ploidy

X_train, X_test, y_train, y_test = train_test_split(fifty_driver_matrix_w_ploidy, y_ploidy, test_size = 0.4, random_state = 0, stratify = y_ploidy)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'C': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

lr_model = LogisticRegression(penalty='l2')

grid_search = GridSearchCV(lr_model, param_grid, scoring='balanced_accuracy')
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))


(236, 51) (158, 51) (236,) (158,)
Best parameters:  {'C': 0.1}
Best score:  0.5030241935483871
Predictions:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0]
Actual:  [1 1 1 1 0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 1 1 1 0 0 1 1 0
 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1
 0 1 1 0 0 1 0 0 1 1 1 0 0 0 0 1 1 0 0 0 0 1 0 0 1 0 1 1 0 0 1 0 1 0 0 0 0
 0 0 1 1 1 1 1 1 1 0]
Balanced Accuracy:  0.504451566951567
ROC AUC:  0.42325498575498577
               precision    recall  f1-score   support

No Recurrence       0.66      0.99      0.79       104
   Recurrence       0.50      0.02      0.04        54

   

In [32]:
# Neural network/Multi-layer perceptron (MLP) algorithm, 50 driver mutations with ploidy

X_train, X_test, y_train, y_test = train_test_split(fifty_driver_matrix_w_ploidy, y_ploidy, test_size = 0.4, random_state = 0, stratify = y_ploidy)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'hidden_layer_sizes': [(1,), (2,), (5,), (10,), (15,)],
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

clf = MLPClassifier(solver='lbfgs', random_state=1, max_iter=2000)

grid_search = GridSearchCV(clf, param_grid, scoring='balanced_accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(236, 51) (158, 51) (236,) (158,)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network

Best parameters:  {'alpha': 0.001, 'hidden_layer_sizes': (1,)}
Best score:  0.5687737191650853
Predictions:  [0 1 1 0 1 0 0 0 1 1 1 0 1 0 0 1 1 1 0 1 0 0 1 0 1 0 0 0 1 0 0 1 1 1 0 0 1
 1 1 0 0 0 1 1 1 1 0 0 0 1 1 0 0 1 0 0 0 1 1 1 1 1 1 0 1 0 1 0 1 1 0 0 1 0
 1 0 1 1 0 1 1 0 1 0 0 1 0 1 1 1 1 1 1 1 1 0 0 1 0 1 0 0 1 0 0 0 1 1 0 1 1
 0 1 0 0 0 0 1 0 1 0 1 0 1 1 1 1 0 0 1 0 1 1 1 1 0 1 1 1 0 1 0 0 1 1 1 1 1
 0 1 0 0 0 0 1 1 0 1]
Actual:  [1 1 1 1 0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 1 1 1 0 0 1 1 0
 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1
 0 1 1 0 0 1 0 0 1 1 1 0 0 0 0 1 1 0 0 0 0 1 0 0 1 0 1 1 0 0 1 0 1 0 0 0 0
 0 0 1 1 1 1 1 1 1 0]
Balanced Accuracy:  0.4100783475783476
ROC AUC:  0.4123931623931624
               precision    recall  f1-score   support

No Recurrence       0.57      0.39      0.47       104
   Recurrence       0.27      0.43      0.33        54

   

In [34]:
# Logsitic Regression, 20 mutations with ploidy

X_train, X_test, y_train, y_test = train_test_split(twenty_gene_matrix_w_ploidy, y_ploidy, test_size = 0.4, random_state = 0, stratify = y_ploidy)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'C': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

lr_model = LogisticRegression(penalty='l2')

grid_search = GridSearchCV(lr_model, param_grid, scoring='balanced_accuracy')
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(236, 21) (158, 21) (236,) (158,)
Best parameters:  {'C': 100}
Best score:  0.5202680265654649
Predictions:  [0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 1 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 1 0 0 0 0 0]
Actual:  [1 1 1 1 0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 1 1 1 0 0 1 1 0
 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1
 0 1 1 0 0 1 0 0 1 1 1 0 0 0 0 1 1 0 0 0 0 1 0 0 1 0 1 1 0 0 1 0 1 0 0 0 0
 0 0 1 1 1 1 1 1 1 0]
Balanced Accuracy:  0.5445156695156695
ROC AUC:  0.5286680911680912
               precision    recall  f1-score   support

No Recurrence       0.68      0.90      0.78       104
   Recurrence       0.50      0.19      0.27        54

   

In [35]:
# Neural network/Multi-layer perceptron (MLP) algorithm, 20 mutations with ploidy

X_train, X_test, y_train, y_test = train_test_split(twenty_gene_matrix_w_ploidy, y_ploidy, test_size = 0.4, random_state = 0, stratify = y_ploidy)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'hidden_layer_sizes': [(1,), (2,), (5,), (10,), (15,)],
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

clf = MLPClassifier(solver='lbfgs', random_state=1, max_iter=2000)

grid_search = GridSearchCV(clf, param_grid, scoring='balanced_accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(236, 21) (158, 21) (236,) (158,)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network

Best parameters:  {'alpha': 1, 'hidden_layer_sizes': (15,)}
Best score:  0.5848553130929791
Predictions:  [0 1 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1
 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0
 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 1 0 1 0 0 0 0 0 0 0 1 1
 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 1 0 1 0 0 0 1]
Actual:  [1 1 1 1 0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 1 1 1 0 0 1 1 0
 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1
 0 1 1 0 0 1 0 0 1 1 1 0 0 0 0 1 1 0 0 0 0 1 0 0 1 0 1 1 0 0 1 0 1 0 0 0 0
 0 0 1 1 1 1 1 1 1 0]
Balanced Accuracy:  0.5286680911680912
ROC AUC:  0.49946581196581197
               precision    recall  f1-score   support

No Recurrence       0.67      0.80      0.73       104
   Recurrence       0.40      0.26      0.31        54

     

In [36]:
# Logsitic Regression, 20 driver mutations with ploidy

X_train, X_test, y_train, y_test = train_test_split(twenty_driver_matrix_w_ploidy, y_ploidy, test_size = 0.4, random_state = 0, stratify = y_ploidy)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'C': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

lr_model = LogisticRegression(penalty='l2')

grid_search = GridSearchCV(lr_model, param_grid, scoring='balanced_accuracy')
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(236, 21) (158, 21) (236,) (158,)
Best parameters:  {'C': 10}
Best score:  0.5281309297912713
Predictions:  [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 1 0
 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0]
Actual:  [1 1 1 1 0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 1 1 1 0 0 1 1 0
 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1
 0 1 1 0 0 1 0 0 1 1 1 0 0 0 0 1 1 0 0 0 0 1 0 0 1 0 1 1 0 0 1 0 1 0 0 0 0
 0 0 1 1 1 1 1 1 1 0]
Balanced Accuracy:  0.4786324786324786
ROC AUC:  0.4513888888888889
               precision    recall  f1-score   support

No Recurrence       0.65      0.85      0.73       104
   Recurrence       0.27      0.11      0.16        54

    

In [37]:
# Neural network/Multi-layer perceptron (MLP) algorithm, 20 driver mutations with ploidy

X_train, X_test, y_train, y_test = train_test_split(twenty_driver_matrix_w_ploidy, y_ploidy, test_size = 0.4, random_state = 0, stratify = y_ploidy)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'hidden_layer_sizes': [(1,), (2,), (5,), (10,), (15,)],
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

clf = MLPClassifier(solver='lbfgs', random_state=1, max_iter=2000)

grid_search = GridSearchCV(clf, param_grid, scoring='balanced_accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(236, 21) (158, 21) (236,) (158,)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network

Best parameters:  {'alpha': 0.1, 'hidden_layer_sizes': (1,)}
Best score:  0.5697462049335863
Predictions:  [0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 1
 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0
 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 1 0 0 0 1 0 0 1 1
 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 1 0 0 1 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0]
Actual:  [1 1 1 1 0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 1 1 1 0 0 1 1 0
 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1
 0 1 1 0 0 1 0 0 1 1 1 0 0 0 0 1 1 0 0 0 0 1 0 0 1 0 1 1 0 0 1 0 1 0 0 0 0
 0 0 1 1 1 1 1 1 1 0]
Balanced Accuracy:  0.5053418803418803
ROC AUC:  0.48370726495726496
               precision    recall  f1-score   support

No Recurrence       0.66      0.79      0.72       104
   Recurrence       0.35      0.22      0.27        54

    

In [38]:
# Logsitic Regression, 10 driver mutations with ploidy

X_train, X_test, y_train, y_test = train_test_split(ten_driver_matrix_w_ploidy, y_ploidy, test_size = 0.4, random_state = 0, stratify = y_ploidy)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'C': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

lr_model = LogisticRegression(penalty='l2')

grid_search = GridSearchCV(lr_model, param_grid, scoring='balanced_accuracy')
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(236, 11) (158, 11) (236,) (158,)
Best parameters:  {'C': 1}
Best score:  0.5416389943074004
Predictions:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0]
Actual:  [1 1 1 1 0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 1 1 1 0 0 1 1 0
 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1
 0 1 1 0 0 1 0 0 1 1 1 0 0 0 0 1 1 0 0 0 0 1 0 0 1 0 1 1 0 0 1 0 1 0 0 0 0
 0 0 1 1 1 1 1 1 1 0]
Balanced Accuracy:  0.48931623931623935
ROC AUC:  0.4565527065527065
               precision    recall  f1-score   support

No Recurrence       0.65      0.92      0.76       104
   Recurrence       0.27      0.06      0.09        54

    

In [39]:
# Neural network/Multi-layer perceptron (MLP) algorithm, 10 driver mutations with ploidy

X_train, X_test, y_train, y_test = train_test_split(ten_driver_matrix_w_ploidy, y_ploidy, test_size = 0.4, random_state = 0, stratify = y_ploidy)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'hidden_layer_sizes': [(1,), (2,), (5,), (10,), (15,)],
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

clf = MLPClassifier(solver='lbfgs', random_state=1, max_iter=2000)

grid_search = GridSearchCV(clf, param_grid, scoring='balanced_accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(236, 11) (158, 11) (236,) (158,)


Best parameters:  {'alpha': 0.0001, 'hidden_layer_sizes': (5,)}
Best score:  0.5602348197343454
Predictions:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0]
Actual:  [1 1 1 1 0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 1 1 1 0 0 1 1 0
 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1
 0 1 1 0 0 1 0 0 1 1 1 0 0 0 0 1 1 0 0 0 0 1 0 0 1 0 1 1 0 0 1 0 1 0 0 0 0
 0 0 1 1 1 1 1 1 1 0]
Balanced Accuracy:  0.5
ROC AUC:  0.4310007122507123
               precision    recall  f1-score   support

No Recurrence       0.66      1.00      0.79       104
   Recurrence       0.00      0.00      0.00        54

     accuracy    

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  